In [ ]:
#install pyspark library
!pip install pyspark

In [ ]:
#import pyspark
import pyspark

In [ ]:
#import sparksession 
from pyspark.sql import SparkSession

In [ ]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.appName("optimization").getOrCreate()

In [ ]:
#To create dataframe form External datasets
AirlineDF = spark.read.option("header", "true").csv("/Users/krishnapratap/Desktop/partation/data/*")

# use take() in place of collect() for reduce time


In [ ]:
%time AirlineDF.take(5)

In [ ]:
%time AirlineDF.show(5)

In [ ]:
# calculate total rows in dataframe
%time AirlineDF.count()

In [ ]:
#calculate total columns in dataframe
len(AirlineDF.columns)

#  Catalyst optimizer 

Catalyst Optimizer improves developer productivity and the performance of their written queries.

In [ ]:
%time AirlineDF.select('OriginCityName') \
.filter("Distance >= 200").filter("ArrDelayMinutes  >= 5").filter("Year  >= 2005").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total Delay Flights").orderBy("OriginCityName").show()

In [ ]:
AirlineDF.select('OriginCityName') \
.filter("Distance >= 200").filter("ArrDelayMinutes  >= 5").filter("Year  >= 2005").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total Delay Flights").orderBy("OriginCityName").explain(mode="formatted")

In [ ]:
from pyspark.sql.functions import col

In [ ]:
%time AirlineDF.select('OriginCityName') \
.orderBy("OriginCityName").withColumnRenamed("count", "Total Delay Flights").filter((col("Year") >= "2005") & (col("ArrDelayMinutes") >= "5") & (col("Distance") >= "200")).groupBy('OriginCityName') \
.count().orderBy("OriginCityName").withColumnRenamed("count", "Total Delay Flights").show()

In [ ]:
AirlineDF.select('OriginCityName') \
.orderBy("OriginCityName").withColumnRenamed("count", "Total Delay Flights").filter((col("Year") >= "2005") & (col("ArrDelayMinutes") >= "5") & (col("Distance") >= "200")).groupBy('OriginCityName') \
.count().orderBy("OriginCityName").withColumnRenamed("count", "Total Delay Flights").explain(mode="formatted")

explain(mode="simple") shows physical plan.

explain(mode="extended") presents physical and logical plans.

explain(mode="codegen") shows the java code planned to be executed.

explain(mode="cost") presents the optimized logical plan and related statistics (if they exist).

explain(mode="formatted") shows a split output created by an optimized physical plan outline, and a section of every node detail.

# When data is huge otherwise not


# use coalesce() in place of repartition() to reduce the no. of partition 

In [ ]:
#to check how many partation in current AirlineDF dataframe
AirlineDF.rdd.getNumPartitions()

In [ ]:
AirlineDF1 = AirlineDF.repartition(2)

In [ ]:
AirlineDF1.rdd.getNumPartitions()

In [ ]:
AirlineDF2 = AirlineDF.coalesce(2)

In [ ]:
AirlineDF2.rdd.getNumPartitions()

# when we load data from other source like s3, database

# when you dealing with heavy-weighted initialization on larger datasets.

# select needed columns from entire dataset and write into new file then create new dataframe

In [ ]:
from pyspark import SparkContext
# Create an RDD
sc = SparkContext()

In [ ]:
df=sc.textFile("/Users/krishnapratap/Desktop/partation/data/*")

In [ ]:
Ardd = df.map(lambda x: x.replace(', ',' ').split(','))

In [ ]:
AirlineRdd=Ardd.map(lambda x: Row(Year=x[0],UniqueCarrier=x[6]))

In [ ]:
from pyspark import SQLContext
from pyspark.sql import Row

In [ ]:
# setting up the sql context
sqlContex = SQLContext(sc)

In [ ]:
AirlineDf1 = sqlContex.createDataFrame(AirlineRdd)

In [ ]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
AirlineDf1.write.option("header", "true").csv("flight/airdata")

In [ ]:
#import sparksession 
from pyspark.sql import SparkSession

In [ ]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.appName("optimization").getOrCreate()

In [ ]:
#To create dataframe form External datasets
AirlineDf2 = spark.read.option("header", "true").csv("flight/airdata/*")

In [ ]:
AirlineDf2.show()

In [ ]:
%time AirlineDf2.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

# Apache Parquet is a columnar file format that provides optimizations to speed up queries and is a far more efficient file format than CSV or JSON, supported by many data processing systems.

In [ ]:
AirlineDf2.write.parquet("parquet/")

In [ ]:
NewDfParquet = spark.read.parquet("parquet/*")

In [ ]:
%time NewDfParquet.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

# Using cache() and persist() methods, Spark provides an optimization mechanism to store the intermediate computation of a Spark DataFrame so they can be reused in subsequent actions.

# Note - When NOT to use Cache/Persist- When the size of the data is large and there are multiple dfs available for cache.

In [ ]:
# before rdd is persisted
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

In [ ]:
from pyspark import StorageLevel

In [ ]:
# persist dataframe to memory and disk
AirlineDf1.persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

In [ ]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
#unpersist dataframe
AirlineDf1.unpersist()

In [ ]:
#persist dataframe to memory 
AirlineDf1.persist(StorageLevel.MEMORY_ONLY)

In [ ]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

In [ ]:
#unpersist dataframe
AirlineDf1.unpersist()

In [ ]:
#persist dataframe to disk
AirlineDf1.persist(StorageLevel.DISK_ONLY)

In [ ]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

In [ ]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
#unpersist dataframe
AirlineDf1.unpersist()

In [ ]:
#cache dataframe
AirlineDf1.cache()

In [ ]:
%time AirlineDf1.select("Year").groupby("Year").count().show()

In [ ]:
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
#unchache dataframe 
AirlineDf1.unpersist()

In [ ]:
# after rdd is uncached
%time AirlineDf1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

# use reduceByKey() in place of Groupbykey

In [ ]:
rdd1=sc.textFile("/Users/krishnapratap/Desktop/partation/data/*")

In [ ]:
# Split words using flatMap
rdd_word = rdd1.flatMap(lambda x: x.split(","))

In [ ]:
# Create a paired-rdd
rdd_pair = rdd_word.map(lambda x: (x, 1))

In [ ]:
# Count occurence per word using groupbykey()
rdd_group = rdd_pair.groupByKey()
rdd_group_count = rdd_group.map(lambda x:(x[0], len(x[1])))
%time rdd_group_count.collect()

In [ ]:
# Count occurence per word using reducebykey()
rdd_reduce = rdd_pair.reduceByKey(lambda x,y: x+y)
%time rdd_reduce.collect()

# use apache arrow to optimize query time 

# create dataframe in pyspark from pandas

In [ ]:
!pip install pyarrow

# here we are creating pandas dataframe from multiple csv file
# so we need to import glob

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pdf1 = pd.read_csv('/Users/krishnapratap/Desktop/partation/data/On_Time_On_Time_Performance_2005_1.csv')

# how to enable arrow for fast pyspark dataframe creation from pandas dataframe

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
%time df2 = spark.createDataFrame(pdf1)

# how to disable arrow  

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false") 

In [ ]:
#we need to cast all the columns in the pandas df to string type to overcome this datatype issue converting pandas df to spark df
%time df1 = spark.createDataFrame(pdf1.astype(str))